# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [22]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    city = []
    adress = []
    postal_code = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])   
    for i in response:
        postal_code.append(i["venue"]["location"]["formattedAddress"][3])    
    response2 = [name , type_, cc, postal_code, adress,  lat, lng ]

    return response2

In [13]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country",  3:"Postal Code", 4:"Adress", 5:"Latitude", 6:"Longitude", })
    return df1

In [14]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country","Postal Code", "Adress","Latitude","Longitude","location"]]
    return df_terminado

# Satisfying Conditions (DATAFRAMES)

## LONDON

## Schools

In [17]:
ldn_Nursery_sch = df_final(url, london_loc, "Nursery School")

In [23]:
ldn_Elementary_sch = df_final(url, london_loc, "High School")

IndexError: list index out of range

In [19]:
llamada_API(url, london_loc, "High School")

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b4b8cb7f964a520f09f26e3',
   'name': 'City of London School for Girls',
   'contact': {},
   'location': {'address': 'St. Giles Terrace',
    'crossStreet': 'Next to Barbican Centre',
    'lat': 51.51911804673561,
    'lng': -0.09432409144499908,
    'labeledLatLngs': [{'label': 'display',
      'lat': 51.51911804673561,
      'lng': -0.09432409144499908}],
    'distance': 534,
    'postalCode': 'EC2Y 8BB,',
    'cc': 'GB',
    'city': 'London',
    'state': 'Greater London',
    'country': 'United Kingdom',
    'formattedAddress': ['St. Giles Terrace (Next to Barbican Centre)',
     'London',
     'Greater London',
     'EC2Y 8BB,',
     'United Kingdom']},
   'categories': [{'id': '4bf58dd8d48988d13d941735',
     'name': 'High School',
     'pluralName': 'High Schools',
     'shortName': 'High School',
     'icon': {'p